In [46]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


# Carregamento e limpeza dos dados

In [47]:
dados = pd.read_csv("diabetes_prediction_dataset.csv")
dados.head(10)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
5,Female,20.0,0,0,never,27.32,6.6,85,0
6,Female,44.0,0,0,never,19.31,6.5,200,1
7,Female,79.0,0,0,No Info,23.86,5.7,85,0
8,Male,42.0,0,0,never,33.64,4.8,145,0
9,Female,32.0,0,0,never,27.32,5.0,100,0


Inicialmente, precisamos entender os dados apresentados:
1) Gender: sexo biológico do indivíduo
2) Age: idade do indivíduo
3) Hypertension: se o paciente foi diagnosticado com hipertensão anteriormente
4) Hert disease: se o paciente possui algum problema de saúde diagnosticado
5) Smoking history: a relação do paciente com o cigarro
6) BMI: indíce de massa corporal
7) HbA1c level: média da quantidade de açucar no sangue do paciente nos últimos 3 meses
8) Blood glucose level: quantidade de glicose no sangue do paciente no momento da última medição
9) Diabetes: se o paciente possui ou não diagnóstico de diabetes

In [48]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [49]:
def sexo_binario(sexo):
    if sexo == 'Female':
        return 0
    else:
        return 1
    
dados['gender'] = dados['gender'].apply(sexo_binario)
dados['age'] = dados['age'].astype(int)

dados.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,80,0,1,never,25.19,6.6,140,0
1,0,54,0,0,No Info,27.32,6.6,80,0
2,1,28,0,0,never,27.32,5.7,158,0
3,0,36,0,0,current,23.45,5.0,155,0
4,1,76,1,1,current,20.14,4.8,155,0


In [50]:
dados['smoking_history'].unique()

array(['never', 'No Info', 'current', 'former', 'ever', 'not current'],
      dtype=object)

In [51]:
smoking_history_mapping = {'never': 0, 'No Info': -1, 'current': 2, 'former': 1, 'ever': 2, 'not current': 0}
dados['smoking_history'] = dados['smoking_history'].map(smoking_history_mapping)

In [52]:
dados.rename(columns = {'gender': 'sexo',
                        'age': 'idade',
                        'hypertension': 'hipertensao',
                        'heart_disease': 'doenca_cardiaca',
                        'smoking_history': 'historico_de_fumo',
                        'bmi': 'imc',
                        'HbA1c_level': 'A1C',
                        'blood_glucose_level': 'glicose_atual',
                        'glucose': 'glicose',
                        'diabetes': 'diabetes'}, inplace = True)

dados

,sexo,idade,hipertensao,doenca_cardiaca,historico_de_fumo,imc,A1C,glicose_atual,diabetes
0,0,80,0,1,0,25.19,6.6,140,0
1,0,54,0,0,-1,27.32,6.6,80,0
2,1,28,0,0,0,27.32,5.7,158,0
3,0,36,0,0,2,23.45,5.0,155,0
4,1,76,1,1,2,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,0,80,0,0,-1,27.32,6.2,90,0
99996,0,2,0,0,-1,17.37,6.5,100,0
99997,1,66,0,0,1,27.83,5.7,155,0
99998,0,24,0,0,0,35.42,4.0,100,0


# Análise exploratória

Podemos começar analisando a relação entre diagnóstico de diabetes e as variáveis categóricas: sexo, hipertensão, doença cardíaca e as categorias ligadas ao fumo.

In [53]:
def porcentagem_por_categoria(categoria):
    categoria_0 = (dados[dados[categoria] == 0]['diabetes'].sum() / dados['diabetes'].sum()) * 100
    categoria_1 = (dados[dados[categoria] == 1]['diabetes'].sum() / dados['diabetes'].sum()) * 100

    print("Incidência de diabetes segundo a categoria: {}".format(categoria))
    print("Valores positivos: {}%\nValores negativos {}%\n".format(round(categoria_1, 1), round(categoria_0, 1)))

In [54]:
porcentagem_por_categoria('sexo')
porcentagem_por_categoria('hipertensao')
porcentagem_por_categoria('doenca_cardiaca')

Incidência de diabetes segundo a categoria: sexo
Valores positivos: 47.5%
Valores negativos 52.5%

Incidência de diabetes segundo a categoria: hipertensao
Valores positivos: 24.6%
Valores negativos 75.4%

Incidência de diabetes segundo a categoria: doenca_cardiaca
Valores positivos: 14.9%
Valores negativos 85.1%



Essa análise deve ser feita com muita cautela. Os dados não estão balanceados em nenhuma dessas categorias, exceto sexo. Ou seja, em todas elas, a incidência de valores negativos e positivos não são proporcionais e, portanto, as médias de incidência de diabetes em cada caso não representam exatamentem se esses parâmetros são fatores de risco ou não.

Para fazer essa análise, é necessário o uso de ferramental estatístico mais avançado que não é o escopo desse projeto. Futuramente essa análise pode ser feita, determinando assim quais desses fatores representam fatores de risco ou não para diabetes, mas isso, no momento, não será feito.

# Modelo de ML

In [55]:
X = dados[['sexo', 'idade', 'hipertensao', 'doenca_cardiaca', 'imc', 'A1C', 'glicose_atual', 'historico_de_fumo']]
y = dados['diabetes']

x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.3)

## Regressor logístico

In [56]:
regressor_logistico = LogisticRegression().fit(x_treino, y_treino)

y_previsto_regressor_logistico = regressor_logistico.predict(x_teste)
acuracia_regressor_logistico = regressor_logistico.score(X, y) * 100

print("Acurácia regressor logístico: {}%".format(round(acuracia_regressor_logistico, 2)))

Acurácia regressor logístico: 95.89%


/home/marcos/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Linear Support vector machine

In [57]:
linear_svm = SVC(kernel = 'linear')
linear_svm.fit(x_treino, y_treino)

y_previsto_linear_svm = linear_svm.predict(x_teste)
acuracia_linear_svm = linear_svm.score(X, y) * 100

print("Acurácia linear support vector machine: {}%".format(round(acuracia_linear_svm, 2)))

Acurácia linear support vector machine: 96.01%
